<a href="https://colab.research.google.com/github/HassanSuhaib71/Learn-Langgraph-With-Hassan/blob/main/17_module_2_6_2_chatbot_external_memory_with_postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai


In [ ]:

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:

import os
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

In [ ]:

from google.colab import userdata
DB_URI = userdata.get('DB_URI')

In [ ]:

from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  GEMINI_API_KEY)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="hi! I'm Hassan")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 2
================================== Ai Message ==================================

Hi Hassan!  Nice to meet you. How can I help you today?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Hassan", additional_kwargs={}, response_metadata={}, id='20bc8f67-d9bd-419c-bd02-06a6269f982d'), AIMessage(content='Hi Hassan!  Nice to meet you. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a6b8bcbb-32b4-4229-b1cf-e0dfdc580a88-0', usage_metadata={'input_tokens': 7, 'output_tokens': 17, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})]}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efdebed-bd1b-6e75-8001-3b2966d9f56f'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': AIMessage(content='Hi Hassan!  Nice to meet you. How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [], 'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}}, 

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="I like painting pictures.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 5
================================== Ai Message ==================================

That's wonderful!  What kind of pictures do you like to paint?  Do you have a favorite subject or style?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Hassan", additional_kwargs={}, response_metadata={}, id='20bc8f67-d9bd-419c-bd02-06a6269f982d'), AIMessage(content='Hi Hassan!  Nice to meet you. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a6b8bcbb-32b4-4229-b1cf-e0dfdc580a88-0', usage_metadata={'input_tokens': 7, 'output_tokens': 17, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like painting pictures.', additional_kwargs={}, response_metadata={}, id='6060776b-8766-4fda-9a1e-20421eb61df2'), HumanMessage(content='I like painting pictures.', additional_kwargs={}, response_metadata={}, id='979b2b4b-7811-47cd-a5c8-c7d9e6f2b6f1'), AIMessage(content="That's wonderful!  What kind of pictures do you like to paint?  Do you have a favorite subject or style?", additional_kwargs={}, response_metadata

In [ ]:

# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="What's my name and what is my hobby?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 7
Messages before summarizing: 7
Existing summary: 
New summary: The conversation began with Hassan introducing himself.  He then stated that he enjoys painting pictures.  Finally, he asked for a summary of the conversation, which includes his name and hobby.
Messages after truncation: 5
================================== Ai Message ==================================

Your name is Hassan, and your hobby is painting pictures.


StateSnapshot(values={'messages': [HumanMessage(content="What's my name and what is my hobby?", additional_kwargs={}, response_metadata={}, id='19d13138-3074-423c-ae87-82a3ffc6a961'), AIMessage(content='Your name is Hassan, and your hobby is painting pictures.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4d0f8cfc-c044-43e1-a7c4-9c8253fb602b-0', usage_metadata={'input_tokens': 75, 'output_tokens': 13, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}})], 'summary': 'The conversation began with Hassan introducing himself.  He then stated that he enjoys painting pictures.  Finally, he asked for a summary of the conversation, which includes his name and hobby.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efdebf0-9b38-6e60-800a-d8b7a80c428a'}}, metadata={'step': 10, 'source': 'loop', 'writes': {'summarize_conversatio

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="Can you describe about abstract paintings?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: The conversation began with Hassan introducing himself.  He then stated that he enjoys painting pictures.  Finally, he asked for a summary of the conversation, which includes his name and hobby.
Message count: 4
================================== Ai Message ==================================

Abstract painting is a genre of art that departs from the realistic representation of the external world. Instead of depicting objects in a recognizable way, abstract art uses shapes, colors, forms, and gestural marks to create a visual experience that's often non-representational.  There's no single definition, and the style encompasses a vast range of approaches.

Here are some key characteristics:

* **Non-representational:**  Abstract art doesn't aim to depict something specific from the real world.  It's about the visual elements themselves, rather than their symbolic meaning or realistic depiction.

* **Emphasis on form and color:**  The primary focus is on the interplay of sh

StateSnapshot(values={'messages': [HumanMessage(content="What's my name and what is my hobby?", additional_kwargs={}, response_metadata={}, id='19d13138-3074-423c-ae87-82a3ffc6a961'), AIMessage(content='Your name is Hassan, and your hobby is painting pictures.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4d0f8cfc-c044-43e1-a7c4-9c8253fb602b-0', usage_metadata={'input_tokens': 75, 'output_tokens': 13, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Can you describe about abstract paintings?', additional_kwargs={}, response_metadata={}, id='82d94f22-7a5f-4cce-bd1e-6d8f45cc2a94'), AIMessage(content='Abstract painting is a genre of art that departs from the realistic representation of the external world. Instead of depicting objects in a recognizable way, abstract art uses shapes, colors, forms, and gestural marks to create a visual exper

In [ ]:

# Retrieve state using thread ID
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Wania", additional_kwargs={}, response_metadata={}, id='be9c43b1-0ee8-4335-9f6a-fae92f6bd5ac'), AIMessage(content='Hi Wania! Nice to meet you.  How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5ae83b0d-0cef-4e1e-ac9d-528d1534bcfc-0', usage_metadata={'input_tokens': 8, 'output_tokens': 18, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efdebeb-44c6-69f6-8001-fd9ea3211d40'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': AIMessage(content='Hi Wania! Nice to meet you.  How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [], 'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}}, id=

In [ ]:
pool.close()
